In [41]:
import os
import pandas as pd
from pyspark.sql import SparkSession

In [42]:
spark = SparkSession.builder \
    .appName("CoffeeData") \
    .master("local[*]") \
    .getOrCreate()

spark

In [43]:
import pandas as pd

df_pd = pd.read_csv(
    r"C:\Users\HP\Desktop\Coffee\data\cleaned\coffee-cleaned.csv",
    sep=';',             
    encoding='utf-8',   
    header =0
)
print(df_pd.shape)
df_pd.head()

(2215, 14)


,coffee_name,total_score,roaster_location,coffee_origin,roast_level,est._price,agtron_ground,agtron_roast,price_per_100g,price_per_ounce,origin_country,clean_roaster_location,roaster_country,origin_country_clean
0,Colombia Finca Campo Hermosa,94.0,"Cleveland, Tennessee","Quindio Department, Colombia",Light,$29.99/8 ounces,62.0,82.0,13.22,29.990000,Colombia,"Cleveland, Tennessee",USA,Colombia
1,Colombia Finca La Sirena Mango Co-Ferment,94.0,"Cleveland, Tennessee","Quindio Department, Colombia",Light,$22.99/8 ounces,63.0,87.0,10.14,22.990000,Colombia,"Cleveland, Tennessee",USA,Colombia
2,In Bloom,94.0,"Jersey City, New Jersey","Colombia, Ethiopia",Light,$25.00/250 grams,64.0,88.0,10.00,0.881834,Ethiopia,"Jersey, New Jersey",USA,Ethiopia
3,Ethiopia Washed Kaffa Gimbo Lot Rich Espresso,96.0,"Chia-Yi, Taiwan","Gimbo, Kaffa Province, Ethiopia",Medium Light,NT $250/8 ounces,55.0,77.0,110.23,250.000000,Ethiopia,"Chia-Yi, Taiwan",Taiwan,Ethiopia
4,Ethiopia Natural Gute Bona,95.0,"Chia-Yi, Taiwan","Sidamo growing region, southern Ethiopia",Medium Light,NT $400/8 ounces,62.0,78.0,176.37,400.000000,Ethiopia,"Chia-Yi, Taiwan",Taiwan,Ethiopia


In [44]:
#df_pyspark = spark.read.csv('coffee-cleaned.csv', sep = ';')
df = spark.read.option("header", True) \
               .option("sep", ";") \
               .option("encoding", "UTF-8") \
               .csv("coffee-cleaned.csv")

df.printSchema()
type(df)

root
 |-- coffee_name: string (nullable = true)
 |-- total_score: string (nullable = true)
 |-- roaster_location: string (nullable = true)
 |-- coffee_origin: string (nullable = true)
 |-- roast_level: string (nullable = true)
 |-- est._price: string (nullable = true)
 |-- agtron_ground: string (nullable = true)
 |-- agtron_roast: string (nullable = true)
 |-- price_per_100g: string (nullable = true)
 |-- price_per_ounce: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- clean_roaster_location: string (nullable = true)
 |-- roaster_country: string (nullable = true)
 |-- origin_country_clean: string (nullable = true)



pyspark.sql.classic.dataframe.DataFrame

In [45]:
df = df.drop("roaster_location", "est._price", "price_per_ounce", "origin_country", "clean_roaster_location")
df.show()

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|            10|            USA|            Ethiopia|
|Ethiopia Washed K...|         96|Gimbo, Kaffa Prov...|Medium Light|           55|          77|        110.23|  

In [47]:
df = df.drop("roaster_location", "est._price", "price_per_ounce", "origin_country", "clean_roaster_location")
df.show()

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|            10|            USA|            Ethiopia|
|Ethiopia Washed K...|         96|Gimbo, Kaffa Prov...|Medium Light|           55|          77|        110.23|  

In [48]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

df = df.withColumn("price_per_100g", col("price_per_100g").cast(DoubleType()))
df.show()

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|          10.0|            USA|            Ethiopia|
|Ethiopia Washed K...|         96|Gimbo, Kaffa Prov...|Medium Light|           55|          77|        110.23|  

In [49]:
### описова статистика 
df.groupBy('origin_country_clean').mean('price_per_100g').show()
### India is NULL. Why?
### NULL country

+--------------------+-------------------+
|origin_country_clean|avg(price_per_100g)|
+--------------------+-------------------+
|               Yemen| 122.30839999999998|
|              Rwanda|  68.84200000000001|
|              Taiwan| 403.74500000000006|
|                NULL|  40.40850490196077|
|             Ecuador| 294.39928571428567|
|           Nicaragua|  8.578333333333333|
|                Peru|            168.382|
|       United States| 22.114814814814817|
|               China|              88.11|
|               India|               NULL|
|Democratic Republ...|  5.578333333333333|
|              Panama| 241.71649681528652|
|              Mexico|             99.837|
|           Indonesia|  34.61774193548386|
|           Guatemala| 147.21472972972973|
|            Honduras|            72.6825|
|            Ethiopia| 109.51148423005563|
|              Brazil|  391.9822222222222|
|               Kenya|  81.24762499999993|
|            Tanzania|  36.31058823529412|
+----------

In [18]:
###India. All records
df.filter(col("origin_country_clean") == "India").show()
### No info about price

+--------------------+-----------+--------------------+-----------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin|roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+-----------+-------------+------------+--------------+---------------+--------------------+
|India AA Elkhill ...|         95|Coorg growing reg...|      Light|           66|          85|          NULL|            USA|               India|
|   S.O. India Sitara|         94|      Southern India|     Medium|           45|          67|          NULL|            USA|               India|
+--------------------+-----------+--------------------+-----------+-------------+------------+--------------+---------------+--------------------+



In [55]:
df_cl = df.na.drop(how="any")
df_cl.show()
num_rows = df_cl.count()
num_cols = len(df_cl.columns)
print((num_rows, num_cols))

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|          10.0|            USA|            Ethiopia|
|Ethiopia Washed K...|         96|Gimbo, Kaffa Prov...|Medium Light|           55|          77|        110.23|  

In [56]:
df_cl.groupBy('origin_country_clean') \
       .mean('price_per_100g') \
       .orderBy('avg(price_per_100g)', ascending=False) \
       .show()

+--------------------+-------------------+
|origin_country_clean|avg(price_per_100g)|
+--------------------+-------------------+
|              Brazil|  467.0728571428571|
|              Taiwan| 403.74500000000006|
|             Ecuador| 294.39928571428567|
|              Panama| 241.71649681528652|
|                Peru|            168.382|
|           Guatemala| 147.21472972972973|
|               Yemen| 129.99173913043478|
|            Ethiopia| 110.31951401869155|
|              Mexico|             99.837|
|               China|              88.11|
|               Kenya|  81.24762499999993|
|            Honduras|            72.6825|
|          Costa Rica|  69.06649350649352|
|              Rwanda|  68.84200000000001|
|            Colombia|  67.43263513513516|
|            Tanzania|  38.47562500000001|
|           Indonesia|  34.61774193548386|
|         El Salvador| 30.712500000000006|
|       United States| 22.114814814814817|
|           Nicaragua|  8.578333333333333|
+----------

In [58]:
df_cl.groupBy('origin_country_clean') \
       .max('price_per_100g') \
       .orderBy('max(price_per_100g)', ascending=False) \
       .show()

+--------------------+-------------------+
|origin_country_clean|max(price_per_100g)|
+--------------------+-------------------+
|              Panama|             6000.0|
|             Ecuador|             4000.0|
|              Brazil|             2980.0|
|            Ethiopia|            2557.32|
|            Colombia|             1680.0|
|              Taiwan|             1200.0|
|           Guatemala|             1058.2|
|                Peru|              800.0|
|               Yemen|             793.65|
|               Kenya|             528.63|
|          Costa Rica|             509.25|
|              Mexico|              498.0|
|         El Salvador|             440.92|
|            Tanzania|              399.0|
|              Rwanda|             352.73|
|           Indonesia|             308.64|
|            Honduras|             154.19|
|               China|              88.11|
|       United States|              70.55|
|           Nicaragua|              26.41|
+----------